Fecha: Julio de 2025

Se busca hacer análisis de frecuencias en las columnas de opinión por año, mes y periódico. E implementar modelado de tópicos. Primer método LDA.

In [130]:
!pip install pyLDAvis

In [ ]:
# Librerías
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import re
from spacy.lang.es.stop_words import STOP_WORDS
import spacy


ModuleNotFoundError: No module named 'pyLDAvis'

In [59]:
# Ruta del directorio que contiene los archivos CSV
folder_path = r'C:\Users\karen\Documents\HumanidadesDigitales_git\BDD_Corpus\BDD_CSV'


In [63]:
dataframes = []

for archivo in os.listdir(folder_path):
    if archivo.endswith('.csv'):
        file_path = os.path.join(folder_path, archivo)        
        try:
            df = pd.read_csv(file_path, encoding='latin-1', sep=';')  
            dataframes.append(df)
        except Exception as e:
            print(f'Error al procesar {file_path}: {e}')


In [78]:
# Concatenar todos los DataFrames en uno solo
if dataframes:
    corpus_completo = pd.concat(dataframes, ignore_index=True)

corpus_completo.head()

,Diario,Autor,Fecha,Título,Texto,Vínculo,Diario
0,El Espectador,Alberto Donadio,30 de diciembre de 2017,César Gaviria y César Mondragón,César Gaviria y César Mondragón son expresiden...,https://web.archive.org/web/20180101101206/htt...,NaN
1,El Espectador,Antonio Casale,30 de diciembre de 2017,Más recomendaciones de fin de año,Un libro y dos documentales imperdibles para e...,https://web.archive.org/web/20180101101206/htt...,NaN
2,El Espectador,Gonzalo Hernández,1 de enero de 2018,Fajardo: para nada tibio,"La Coalición Colombia Partido Alianza Verde, ...",https://web.archive.org/web/20180102104221/htt...,NaN
3,El Espectador,Eduardo Barajas Sandoval,1 de enero de 2018,Macedonia de Norte,Las interpretaciones de la historia sirven com...,https://web.archive.org/web/20180102104221/htt...,NaN
4,El Espectador,Daniel Emilio Rojas Castro,1 de enero de 2018,El nacionalismo según Vargas Llosa,La semana pasada Mario Vargas Llosa publicó un...,https://web.archive.org/web/20180102104221/htt...,NaN


In [ ]:
# Preprocesamiento de los datos

# Cargar modelo de lenguaje en español de Spacy, el modelo más pequeño
nlp = spacy.load("es_core_news_sm")

In [ ]:
def preprocesar_texto(texto):
    # Limpieza para eliminar URLs, emails, caracteres especiales
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
    texto = re.sub(r'\S+@\S+', '', texto)
    texto = re.sub(r'[^\w\sáéíóúñÁÉÍÓÚÑ]', ' ', texto)
    
    # Procesamiento con spaCy
    doc = nlp(texto.lower())
    
    # Lematización y filtrado
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop 
        and not token.is_punct 
        and not token.is_space
        and len(token.lemma_) > 2  # Eliminar palabras muy cortas
        and token.lemma_.isalpha()  # Solo palabras alfabéticas
    ]
    
    return " ".join(tokens)

In [121]:
titulos_procesados = corpus_completo['Título'].astype(str).apply(preprocesar_texto)

In [122]:
titulos_procesados

0        césar gavirio césar mondragón
1                    recomendación año
2                        fajardo tibio
3                      macedonia norte
4             nacionalismo vargas llós
                     ...              
13339          capitalismo escandinava
13340          derrotar guacho amazona
13341               porte arma derecho
13342                         congreso
13343           casanarar bicentenario
Name: Título, Length: 13344, dtype: object

In [ ]:
# Vectorización (creación de matriz documento-término)
vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=2000)
X = vectorizer.fit_transform(titulos_procesados)

In [127]:
# Modelado de tópicos con LDA
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)

# Función para mostrar los tópicos
def mostrar_topicos(modelo, vectorizer, n_palabras=10):
    palabras = vectorizer.get_feature_names_out()
    for idx, topico in enumerate(modelo.components_):
        print(f"Tópico #{idx}:")
        print(" ".join([palabras[i] for i in topico.argsort()[:-n_palabras - 1:-1]]))
        print()

mostrar_topicos(lda, vectorizer)

Tópico #0:
paz guerra política gazapera pandemia mujer vida elección pasar niño

Tópico #1:
duque presidente país trump bogotá justicia iván futuro prima ganar

Tópico #2:
colombia democracia tiempo tola maruja derecho educación reforma miedo crisis

Tópico #3:
año político petro gobierno voto mundo economía volver corrupción hora

Tópico #4:
historia nacional fiscal uribir venezuela colombiano ley muerte cambio covid



In [105]:
# Vectorización incluyendo bigrams
vectorizer_bigrams = CountVectorizer(ngram_range=(1, 2), max_df=0.85, min_df=2)
X_bigrams = vectorizer_bigrams.fit_transform(titulos_procesados)

# LDA con bigrams
lda_bigrams = LatentDirichletAllocation(n_components=5, random_state=42)
lda_bigrams.fit(X_bigrams)

mostrar_topicos(lda_bigrams, vectorizer_bigrams)

Tópico #0:
paz petro tiempo tola maruja tola maruja historia venezuela miedo educación

Tópico #1:
político vida bogotá justicia pasar palabra ley matar lección agua

Tópico #2:
duque presidente guerra política gobierno voto covid mujer iván público

Tópico #3:
democracia gazapera economía corrupción muerte memoria jep problema uribir centro

Tópico #4:
colombia país año social trump pandemia niño ganar reforma nacional



In [115]:
corpus_completo.iloc[13343]['Título']  # Ejemplo de título original

'Casanare y el bicentenario'

In [113]:
from gensim.models import CoherenceModel
import numpy as np

# 1. Obtener los términos más importantes por tópico
def get_top_words_lda(model, feature_names, n_top_words):
    top_words = []
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_words.append([feature_names[i] for i in top_features_ind])
    return top_words

# 2. Obtener los nombres de las características (palabras) del vectorizer
feature_names = vectorizer.get_feature_names_out()

# 3. Obtener las palabras más relevantes por tópico (ej: 10 palabras por tópico)
top_words_per_topic = get_top_words_lda(lda, feature_names, n_top_words=10)

# 4. Asegurarse de que los textos están tokenizados correctamente
textos_tokenizados = [texto.split() for texto in titulos_procesados]

# 5. Crear el diccionario de Gensim
dictionary = Dictionary(textos_tokenizados)

# 6. Calcular la coherencia
coherence_model = CoherenceModel(
    topics=top_words_per_topic,  # Lista de listas de palabras
    texts=textos_tokenizados,    # Lista de listas de tokens
    dictionary=dictionary,       # Objeto Dictionary de Gensim
    coherence='c_v'
)

print(f"Coherencia: {coherence_model.get_coherence():.3f}")

Coherencia: 0.441


In [128]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, X, vectorizer)
vis

ModuleNotFoundError: No module named 'pyLDAvis.sklearn'